<a href="https://colab.research.google.com/github/Kriz-827/malayalam-speech-to-text/blob/main/malayalamstt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git transformers datasets


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00


In [ ]:
import zipfile
import os

# Update with your ZIP file name
zip_path = "/content/drive/MyDrive/internship/malayalam_fine_tune.zip"
extract_dir = "/content/drive/MyDrive/malayalam_fine_tune_unzipped"

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(" Extracted to:", extract_dir)


✅ Extracted to: /content/drive/MyDrive/malayalam_fine_tune_unzipped


In [ ]:
import os
import pandas as pd

#  Base paths
BASE_DIR = "/content/drive/MyDrive"
UNZIPPED_DIR = os.path.join(BASE_DIR, "malayalam_fine_tune_unzipped", "malayalam_fine_tune")
OUTPUT_CSV = os.path.join(BASE_DIR, "manifest1000.csv")  # <-- renamed for clarity

AUDIO_EXTENSIONS = [".wav", ".mp3", ".flac"]
samples = []

#  Go through each subfolder
for root, _, files in os.walk(UNZIPPED_DIR):
    audio_files = [f for f in files if any(f.lower().endswith(ext) for ext in AUDIO_EXTENSIONS)]
    text_files = [f for f in files if f.lower().endswith(".txt")]

    if len(audio_files) == 1 and len(text_files) == 1:
        audio_path = os.path.join(root, audio_files[0])
        transcript_path = os.path.join(root, text_files[0])
        try:
            with open(transcript_path, "r", encoding="utf-8") as f:
                transcript = f.read().strip()
                if not transcript:
                    continue
                rel_path = os.path.relpath(audio_path, BASE_DIR).replace("\\", "/")
                samples.append({"audio_path": rel_path, "transcript": transcript})
        except Exception as e:
            print(f" Error reading in {root}: {e}")
    elif len(audio_files) > 1 or len(text_files) > 1:
        print(f" Skipping {root} (multiple audio or text files)")
    else:
        print(f" Skipping {root} (missing audio or transcript)")

print(f" Total valid pairs found: {len(samples)}")

#  Save ALL valid samples
df = pd.DataFrame(samples)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print(f" Saved {len(samples)} samples to: {OUTPUT_CSV}")


⛔ Skipping /content/drive/MyDrive/malayalam_fine_tune_unzipped/malayalam_fine_tune (missing audio or transcript)
✅ Total valid pairs found: 1099
✅ Saved 1099 samples to: /content/drive/MyDrive/manifest1000.csv


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git transformers datasets torchaudio


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!nvidia-smi

Sat Jun 21 06:23:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

#  Use whisper-small instead of tiny
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(BASE_DIR, "finetuned_model_small"),
    per_device_train_batch_size=4,  # Tesla T4 can handle this
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    num_train_epochs=10,
    logging_steps=10,

    save_total_limit=1,
    report_to="none",
    fp16=True  #  Use mixed precision for better performance
)


In [ ]:
import pandas as pd
from datasets import Dataset

CSV_PATH = "/content/drive/MyDrive/manifest1000.csv"

# Step 1: Load using pandas
df = pd.read_csv(CSV_PATH)

# Step 2: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# (Optional) shuffle for training
dataset = dataset.shuffle(seed=42)

print(" Loaded dataset with", len(dataset), "samples")
dataset[0]

 Loaded dataset with 1099 samples


{'audio_path': 'malayalam_fine_tune_unzipped/malayalam_fine_tune/Mal_0050_Mal_f_0022/audio.wav',
 'transcript': 'കുട്ടികളെ കൊണ്ടുവന്നതിന്റെ നടപടിക്രമങ്ങളില്\u200d വീഴ്ചവന്നിട്ടുള്ളതായി മുഖ്യമന്ത്രി വെളിപ്പെടുത്തുകയും പോലീസിന്റെ അന്വേഷണം ആരംഭിക്കുകയുംചെയ്തെങ്കിലും പിന്നീട്\u200c സമ്മര്\u200dദവിഭാഗങ്ങളുടെ ശക്തികൊണ്ട്\u200c കുട്ടിക്കടത്തുകേസ്\u200c മന്ദീഭവിച്ചു.'}

In [ ]:
import os
import torchaudio
from transformers import WhisperProcessor

BASE_DIR = "/content/drive/MyDrive"
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

def prepare_sample(example):
    audio_path = os.path.join(BASE_DIR, example["audio_path"])
    waveform, sr = torchaudio.load(audio_path)

    # Process audio input
    input_features = processor.feature_extractor(
        waveform.squeeze(), sampling_rate=sr, return_tensors="pt"
    ).input_features[0]

    # Process and truncate text tokens
    labels = processor.tokenizer(
        example["transcript"],
        return_tensors="pt",
        padding="longest",
        truncation=True,              #  truncate long transcripts
        max_length=448                #  max allowed by Whisper
    ).input_ids[0]

    return {"input_features": input_features, "labels": labels}

dataset = dataset.map(prepare_sample)


Map:   0%|          | 0/1099 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        batch["labels"] = labels_batch["input_ids"]
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        batch["labels"] = labels_batch["input_ids"]
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    tokenizer=processor.tokenizer  # no deprecation warning now
)

trainer.train()


/tmp/ipython-input-27-1042358833.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
10,0.155500
20,0.119100
30,0.141400
40,0.153400
50,0.145800
60,0.122300
70,0.124000
80,0.138400
90,0.127200
100,0.135900


Step,Training Loss
10,0.155500
20,0.119100
30,0.141400
40,0.153400
50,0.145800
60,0.122300
70,0.124000
80,0.138400
90,0.127200
100,0.135900


TrainOutput(global_step=1380, training_loss=0.03976071913983079, metrics={'train_runtime': 3003.4194, 'train_samples_per_second': 3.659, 'train_steps_per_second': 0.459, 'total_flos': 3.1715535495168e+18, 'train_loss': 0.03976071913983079, 'epoch': 10.0})

In [ ]:
save_path = os.path.join(BASE_DIR, "finetuned_model_small")
model.save_pretrained(save_path)
processor.save_pretrained(save_path)
print("✅ Fine-tuned model saved to:", save_path)


✅ Fine-tuned model saved to: /content/drive/MyDrive/finetuned_model_small


In [ ]:
!pip install flask flask-cors flask-ngrok


In [ ]:
!pip install pyngrok


In [ ]:

from pyngrok import ngrok, conf

conf.get_default().auth_token = "2xzQWPqwi9rSeCjcEJjy6yutCtb_475zbrcX4iMcMvJhYfwKk"


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tempfile, os
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, GenerationConfig
from pyngrok import ngrok


MODEL_PATH = "/content/drive/MyDrive/finetuned_model_small"
print(" Loading fine-tuned Whisper-small model...")

model = WhisperForConditionalGeneration.from_pretrained(MODEL_PATH)
processor = WhisperProcessor.from_pretrained(MODEL_PATH)

model.config.forced_decoder_ids = None
model.generation_config = GenerationConfig()


app = Flask(__name__)
CORS(app)

@app.route("/transcribe", methods=["POST"])
def transcribe():
    print(" Received transcription request")

    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    audio = request.files["audio"]
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp:
        audio.save(temp.name)
        temp_path = temp.name

    try:
        waveform, sr = torchaudio.load(temp_path)
        max_samples = sr * 10  # 10 seconds
        waveform = waveform[:, :max_samples]

        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
            sr = 16000

        inputs = processor.feature_extractor(
            waveform.squeeze(), sampling_rate=sr, return_tensors="pt"
        ).input_features


        forced_ids = processor.get_decoder_prompt_ids(language="ml", task="transcribe")


        generation_config = GenerationConfig(
            forced_decoder_ids=forced_ids,
            decoder_start_token_id=50258,
            max_length=448,
            suppress_tokens=[],
            begin_suppress_tokens=[],
        )

        ids = model.generate(inputs, generation_config=generation_config)
        transcription = processor.tokenizer.batch_decode(ids, skip_special_tokens=True)[0]

        print("📝 Transcription:", transcription)
        return jsonify({"transcription": transcription})

    except Exception as e:
        print(" Error:", str(e))
        return jsonify({"error": str(e)})
    finally:
        os.remove(temp_path)


public_url = ngrok.connect(5000)
print(f" Ngrok Tunnel URL: {public_url}")
app.run(port=5000)


 Loading fine-tuned Whisper-small model...
 Ngrok Tunnel URL: NgrokTunnel: "https://a1e3-35-197-105-198.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, GenerationConfig
import torchaudio

# Load fine-tuned model
model_path = "/content/drive/MyDrive/finetuned_model_small"
model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)

model.config.forced_decoder_ids = None
model.generation_config = GenerationConfig()

# Load and trim audio
file_path = "/content/drive/MyDrive/test2.wav"  # change if needed
waveform, sr = torchaudio.load(file_path)
waveform = waveform[:, :sr * 10]  # trim to 10s

if sr != 16000:
    waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    sr = 16000

# Process
inputs = processor.feature_extractor(
    waveform.squeeze(), sampling_rate=sr, return_tensors="pt"
).input_features

# Generate
forced_ids = processor.get_decoder_prompt_ids(language="ml", task="transcribe")

gen_config = GenerationConfig(
    forced_decoder_ids=forced_ids,
    decoder_start_token_id=50258,
    max_length=448,
    suppress_tokens=[],
    begin_suppress_tokens=[],
)

ids = model.generate(inputs, generation_config=gen_config)

# Decode
transcription = processor.tokenizer.batch_decode(ids, skip_special_tokens=True)[0]
print(" Transcription:", transcription)


 Transcription:  ഒരു ഉത്പന്നം അല്ലെങ്കില്‍ സേവനം തകരാറിലായാല്‍, ഉപവോക്താവ ഫെഡരള്‍ കമ്മീഷനില്‍ പറാദിപ്പെടാം.
